In [61]:
import requests
import urllib
from bs4 import BeautifulSoup

In [62]:
blacklist = [
    "Main_Page",
    "Help:",
    "Special:",
    "Portal:",
    "Talk:",
    "Wikipedia:",
    "Template:"
]

In [64]:
urllib.parse.unquote("/wiki/Hamilton%E2%80%93Jacobi%E2%80%93Bellman_equation")

'/wiki/Hamilton–Jacobi–Bellman_equation'

In [71]:
def parse_wiki_link(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')

    wiki_content_links = set()
    for link in soup.find_all('a', href=True):
        clean_link = link["href"]
        clean_link = urllib.parse.unquote(clean_link)
        if clean_link.startswith("/wiki/"):
            wiki_link = clean_link[6:]
            if any(x in wiki_link for x in blacklist):
                continue
            wiki_content_links.add(clean_link)
    return wiki_content_links

In [72]:
link = "https://en.wikipedia.org/wiki/Hamilton–Jacobi–Bellman_equation"
l = parse_wiki_link(link)

In [60]:
class UserHistory:
    def __init__(self, user_visits):
        # user_vists is a list of links in chronological order ascending
        # user_vists[-1] is the current page
        self.outgoing_links = set()
        for link in user_visits:
            self.outgoing_links.update(parse_wiki_link(link))
        already_visited_pages = {link.split("wikipedia.org/"[1]) for link in user_visits}

        # remove self-loops
        self.outgoing_links -= already_visited_pages

        # first stage of ranking: use NLP to choose top-k links from self.outgoing_links
        #   Collect text from all pages on user_visits
        #   TF-IDF for similarity between (Text in links) and (Text in all pages)
        #   For top textually similar links, crawl these target pages for text & links
        #       check link similarity (can also use text from the target page now)


In [ ]:
def recommend_similar_pages(user_history):
    # user_history  

In [50]:
wiki_content_links = set()
for link in soup.find_all('a', href=True):
    wiki_link = link["href"]
    if wiki_link.startswith("/wiki/"):
        wiki_link = wiki_link[6:]
        if any(x in wiki_link for x in blacklist):
            continue
        wiki_content_links.add(link["href"])

In [51]:
for i in wiki_content_links:
    print(i)

/wiki/Category:Dynamic_programming
/wiki/Category:Optimal_control
/wiki/Backward_induction
/wiki/Multilayer_perceptron
/wiki/Category:All_articles_with_unsourced_statements
/wiki/Pierre-Louis_Lions
/wiki/Template:Cite_journal
/wiki/Pontryagin%27s_maximum_principle
/wiki/Proceedings_of_the_National_Academy_of_Sciences_of_the_United_States_of_America
/wiki/Brachistochrone_problem
/wiki/Category:Stochastic_control
/wiki/Doi_(identifier)
/wiki/S2CID_(identifier)
/wiki/Richard_Bellman
/wiki/Category:William_Rowan_Hamilton
/wiki/Classical_physics
/wiki/ISBN_(identifier)
/wiki/Bellman_equation
/wiki/Subderivative
/wiki/It%C3%B4_calculus
/wiki/Principle_of_optimality
/wiki/Little-o_notation
/wiki/Minimax_solution
/wiki/Merton%27s_portfolio_problem
/wiki/Loss_function
/wiki/Viscosity_solution
/wiki/Hamilton%E2%80%93Jacobi%E2%80%93Bellman_equation
/wiki/Bibcode_(identifier)
/wiki/Dimitri_Bertsekas
/wiki/Necessary_and_sufficient_condition
/wiki/Optimal_control_theory
/wiki/Sum-of-squares_optimiza